# KoSOLAR-v0.2-gugutypus-10.7B
<font color='orange'>LoRA를 활용한 **KoSOLAR-10.7B-v0.2** fine-tuning with **KOR-gugugu-platypus-set**</font>

<br>

<img src = "https://drive.google.com/uc?id=1G_OreIMXjxZNwo9rpJDFHMIslRB1RwOM" height=512, width=512>

**gugutypus** = gugugo + platypus

-------
## Notebook Summary


### 1. KoSOLAR-v0.2-gugutypus-10.7B 📖

- Code 환경설정
- Hyperparameters in LLM
- <font color='#e79531'><b>LoRA 🔥</b></font>
- Instruction input-output dataset
- <font color='#9e95f2'><b>LLM fine-tuning ☀️</b></font> (with **KOR-gugugu-platypus-set** 🕊️)
- <font color='#f2cc3b'><b>Model Upload 🤗</b></font>

  
### 3. 실습에 이용되는 모델과 데이터셋 📖

- ✅ 모델: [yanolja/KoSOLAR-10.7B-v0.2](https://huggingface.co/yanolja/KoSOLAR-10.7B-v0.2)
- ✅ 데이터셋: [kyujinpy/KOR-gugugu-platypus-set](https://huggingface.co/datasets/kyujinpy/KOR-gugugu-platypus-set)


### 4. 참고자료

- Model
  - [upstage/SOLAR-10.7B-v1.0](https://huggingface.co/upstage/SOLAR-10.7B-v1.0)
- Dataset
  - [Platypus](https://github.com/arielnlee/Platypus)  
  - [garage-bAInd/Open-Platypus](https://huggingface.co/datasets/garage-bAInd/Open-Platypus)
  - [squarelike/OpenOrca-gugugo-ko](https://huggingface.co/datasets/squarelike/OpenOrca-gugugo-ko)
- Community
  - [HuggingFace](https://huggingface.co/)  
  - [KO-LLM LeaderBoard](https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard)  
  - [EN-LLM LeaderBoard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)


--------
# ♾️ 환경설정

- 🤗 transformers: LLM fine-tuning 및 전반적인 기능
- 🤗 bitsandbytes: bit 단위로 모델을 불러올 수 있는 기능
- 🤗 accelerate: GPU 가속
- 🔥 loralib: LoRA 세팅
- 🤗 datasets: 데이터셋 활용
- 💎 peft: Parameter Efficient Fine Tuning 적용


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip list

--------
# 👨‍💻 HuggingFace 로그인

1. [HuggingFace 🤗](https://huggingface.co/)에서 자신의 Access token 준비한다.
  - `Role: read` token과 `Role: write` token이 각각 필요

2. 코드에서 HuggingFace login을 한다.
  - ⭐`Role: read`로 생성된 token을 복사 한 후, 아래의 `<read_token> 부분에 붙여넣는다.
```bash
!huggingface-cli login --token <read_token>
```

In [ ]:
# Huggingface login

# READ token
!huggingface-cli login --token <read_token> # your code

--------
# 😎 모듈 불러오기


In [ ]:
import os
import os.path as osp
import sys
import fire
import json
from typing import List, Union

import torch
from torch.nn import functional as F

import transformers
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

from datasets import load_dataset

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_int8_training,
    set_peft_model_state_dict
)
from peft import PeftModel

--------
# 🤗 Base Model (LLM)
- 이용할 base LLM 모델: [yanolja/KoSOLAR-10.7B-v0.2](https://huggingface.co/yanolja/KoSOLAR-10.7B-v0.2)     

## ✔️ Model Access 확인
> ✅ 모델을 이용하기 전에 access가 있는지 꼭 확인해야 한다!  

In [ ]:
#@title 🤗 Base model 선택하기
device = 'auto' #@param {type: "string"}
base_LLM_model = 'yanolja/KoSOLAR-10.7B-v0.2' #@param {type: "string"}

In [ ]:
# 모델 다운로드 (~30분)

model = AutoModelForCausalLM.from_pretrained(
    base_LLM_model,
    load_in_8bit=True, # LoRA
    #load_in_4bit=True, # Quantization Load
    torch_dtype=torch.float16,
    device_map=device)

tokenizer = AutoTokenizer.from_pretrained(base_LLM_model)

In [ ]:
# BOS, EOS, PAD 토큰 확인

# Check special token
bos = tokenizer.bos_token_id # 문장 시작 토큰
eos = tokenizer.eos_token_id # 문장 끝 토큰
pad = tokenizer.pad_token_id # 문장 패딩 토큰
tokenizer.padding_side = "right" # 패딩 오른쪽

print("BOS token:", bos) # 1
print("EOS token:", eos) # 32000
print("PAD token:", pad) # 2

In [ ]:
if (pad == None) or (pad == eos):
    tokenizer.pad_token_id = 0  # 만약 패딩값이 없거나 eos값과 같다면,
print("length of tokenizer:",len(tokenizer)) # 40960

In [ ]:
print(model)

In [ ]:
print(type(model)) # 모델의 타입 확인

--------
# 💡 Hyperparameters setting

1. **cutoff_len**
> 모델에 들어갈 **sequence의 최대 길이를 설정**

2. **warmup_steps**
> warmup_steps은 천천히 learning rate를 올려서, 학습의 불안정성을 방지해주는 역할을 함
  
  - **Example)** learning rate (LR)을 1e-2로 설정했다면, LR을 처음에 초기값보다 낮게 시작하여 100 step에 걸쳐서 LR를 1e-2로 올린다.
  
3. **Optimizer**
> 최근에는 Adam보다 AdamW를 더 많이 이용하는 추세임
> AdamW는 Adam에 비해서 더 <font color='#ea7a7a'><b>general</b></font>하게 성능을 만들어주는 장점이 있음
  
  - Adam vs <font color='#ea7a7a'><b>**AdamW**</b></font>

4. **lr_scheduler**
> lr_scheduler는 learning rate를 훈련하는 동안 조절해주는 옵션
> 아래의 목록에 있는 총 3가지 함수 옵션이 가장 대표적으로 사용됨
  
  - constant
  - linear
  - <font color='#ea7a7a'><b>consine</b></font>

5. **Others**

  - **⭐ Learning rate**
  - Batch size
  - weight_decay
  - max grad norm: gradient vector 크기 조절 (gradient clipping)

## 👨‍💻 LoRA hyperparameters

<img src = "https://drive.google.com/uc?id=1ZoFUOPAyfWx80qk9Oa6b8oY0h1bJRoE0" width=512>

  -  <font color='#e79531'><b>LoRA_r</b></font> : LoRA adapter의 차원을 결정하는 파라미터
  - Lora_alpha : LoRA adapter의 scaling 값을 결정하는 파라미터
  - Lora_dropout : LoRA adapter의 dropout 파라미터
  -  <font color='#e79531'><b>**Lora_target_modules**</b></font> : LoRA adapter를 적용할 layers
  ```
Lora_target_modules 종류:
["embed_tokens", "lm_head", "q_proj", "k_proj", "v_proj", "o_proj", ...] # in llama2
  ```

  <br>
  
### 🔎 LoRA가 적용되는 코드 예제
  
```python
class Linear(nn.Linear, LoRALayer):
  # LoRA implemented in a dense layer
  def __init__(
          self,
          in_features: int,
          out_features: int,
          r: int = 0,
          lora_alpha: int = 1,
          lora_dropout: float = 0.,
          fan_in_fan_out: bool = False, # Set this to True if the layer to replace stores weight like (fan_in, fan_out)
          merge_weights: bool = True,
          **kwargs
      ):
          nn.Linear.__init__(self, in_features, out_features, **kwargs)
          LoRALayer.__init__(self, r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout,
                              merge_weights=merge_weights)
          self.fan_in_fan_out = fan_in_fan_out
          # 실제 훈련할 파라미터
          if r > 0:
              self.lora_A = nn.Parameter(self.weight.new_zeros((r, in_features)))
              self.lora_B = nn.Parameter(self.weight.new_zeros((out_features, r)))
              self.scaling = self.lora_alpha / self.r
              # Freezing the pre-trained weight matrix
              self.weight.requires_grad = False
          self.reset_parameters()
          if fan_in_fan_out:
            self.weight.data = self.weight.data.transpose(0, 1)
```

In [ ]:
# 하이퍼 파라미터

# 데이터셋과 훈련 횟수와 관련된 하이퍼 파라미터
batch_size = 16
num_epochs = 1
micro_batch = 1
gradient_accumulation_steps = batch_size // micro_batch

# 훈련 방법에 대한 하이퍼 파라미터
cutoff_len = 4096
lr_scheduler = 'cosine'
warmup_ratio = 0.06 # warmup_steps = 100
learning_rate = 4e-4
optimizer = 'adamw_torch'
weight_decay = 0.01
max_grad_norm = 1.0 # default: 1.0
neftune_noise_alpha = 15

# LoRA config
lora_r = 8
lora_alpha = 8
lora_dropout = 0.05
lora_target_modules = ["gate_proj", "down_proj", "up_proj"]

# Tokenizer에서 나오는 input값 설정 옵션
train_on_inputs = False
add_eos_token = False

# Others
output_dir = './tuned_LLM'

resume_from_checkpoint = False # !! 만약 모델을 이어서 훈련하고 싶다면, 아래 주석처럼 './custom_LLM/checkpoint-[xxx]'와 같이 파일 경로를 입력해야 함
# resume_from_checkpoint = f"{output_dir}/checkpoint-100" 

--------
# 🤗 Dataset Loading and formatting
> 👨‍💻 LLM fine-tuning을 위한 데이터셋을 불러오고 <font color='green'><b>instruction input-output</b></font>형태로 만들기

- 이용할 데이터셋: [kyujinpy/KOR-gugugu-platypus-set](https://huggingface.co/datasets/kyujinpy/KOR-gugugu-platypus-set)

In [ ]:
#@title 🤗 Choose Dataset
dataset = 'kyujinpy/KOR-gugugu-platypus-set' #@param {type: "string"}

In [ ]:
# 데이터셋 다운로드

data = load_dataset(dataset)
print(data['train'])

## 🤗 Instruction input-output template
> 데이터는 아래와 같은 형식을 갖고 있음
   
### ✅ Input이 있는 example:  
  
```
Input:
당신은 사람들이 정보를 찾을 수 있도록 도와주는 인공지능 비서입니다.
사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하게 대답하는 것입니다.
답변하는 동안 단계별로 생각하고 답변을 정당화하세요.  
  
Instruction:
질문입니다: 시속 60km의 속도로 달리는 기차가 9초 만에 기둥을 통과합니다. 기차의 길이는 얼마입니까?
옵션: (A) 287 (B) 699 (C) 677 (D) 168 (E) 150 먼저 생각해 봅시다. 무작위로 추론해 봅시다:
  
Output:
기차의 길이를 구하려면 기차가 기둥을 통과하는 동안 이동하는 거리를 결정해야 합니다.
1단계: 기차의 속도를 초당 미터로 변환합니다. 1km = 1000미터, 1시간 = 3600초, 따라서 60km/hr = (60 * 1000) / 3600 = 16.67m/s(대략).
2단계: 기차가 9초 동안 이동한 거리를 계산합니다. 거리 = 속도 × 시간 거리 = 16.67m/s * 9초 = 150미터.
3단계: 계산된 거리를 주어진 옵션과 비교합니다.
계산된 거리(150미터)에 가장 가까운 옵션은 옵션 (E) 150입니다.
따라서 열차의 길이는 약 150미터입니다.
```

  
### ✅ Input이 없는 example:

```
Instruction:
질문이 주어집니다: 다음 기사에 대한 객관식 문제를 작성하세요: 기사: 겨울은 무슨 일이 일어날지 알기 어렵고 사고가 너무 쉽게 일어나기 때문에 위험합니다.
안개가 언덕 꼭대기에서 여러분을 기다리고 있을 수 있습니다. 녹는 눈 밑에 얼음이 숨어 있다가 운전자를 도로 밖으로 내보내려고 기다리고 있을 수도 있습니다.
마주 오는 차가 갑자기 도로를 가로질러 미끄러질 수도 있습니다. 빙판길 운전의 첫 번째 규칙은 부드럽게 운전하는 것입니다.
갑작스러운 움직임은 자동차를 제어하기 매우 어렵게 만들 수 있습니다. 따라서 차를 출발하거나 정지할 때마다, 속도를 높이거나 낮출 때마다 가능한 한 부드럽고 천천히 운전해야 합니다.
옆 좌석에 뜨거운 커피 한 잔을 가득 들고 운전한다고 가정해 보겠습니다. 커피를 엎지르지 않도록 운전하세요.
두 번째 규칙은 무슨 일이 일어날지 주의를 기울이는 것입니다. 얼음이 많을수록 도로 아래쪽을 더 잘 살펴야 합니다.
차를 부드럽게 정지하는 데 걸리는 시간을 테스트하세요. 생각보다 더 빨리 운전할 수 있다는 점을 기억하세요.
일반적으로 도로가 젖어 있을 때는 평소 정지 거리의 두 배, 눈이 쌓여 있을 때는 이 거리의 세 배, 빙판길에서는 그보다 더 많은 거리를 확보하세요.
항상 차량을 통제하려고 노력하면 문제가 발생하지 않습니다. 정답은

Output:
빙판길 운전 시 가장 중요한 규칙은 무엇인가요?
A) 위험 지역을 더 빨리 통과하기 위해 속도를 높입니다.
B) 잠재적 위험을 피하기 위한 갑작스러운 움직임
C) 부드럽고 부드럽게 운전하세요
D) 바로 앞 도로에만 집중하기
```

## 🔎 Alpaca template을 활용하여 데이터셋을 instruction input-output 형식으로 재정의
- [Alpaca github](https://github.com/tatsu-lab/stanford_alpaca)

### ✅ 한국어 버전 template
```
{
    "prompt_input": "아래는 작업을 설명하는 지침과 추가 입력을 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 답변을 작성해주세요.\n\n### 지침:\n{instruction}\n\n### 입력:\n{input}\n\n### 답변:\n",
    "prompt_no_input" : "아래는 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 답변을 작성해주세요.\n\n### 지침:\n{instruction}\n\n### 답변:\n",
    "response_split": "### 답변:"
}
```

### ✅ (참고) 영어 버전 template
```
{
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
    "response_split": "### Response:"    
}
```

In [ ]:
# Instruction tuning을 위한 한국어 template 작성.

instruct_template = {
    "prompt_input": "아래는 작업을 설명하는 지침과 추가 입력을 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 답변을 작성해주세요.\n\n### 지침:\n{instruction}\n\n### 입력:\n{input}\n\n### 답변:\n",
    "prompt_no_input" : "아래는 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 답변을 작성해주세요.\n\n### 지침:\n{instruction}\n\n### 답변:\n",
    "response_split": "### 답변:"
}

In [ ]:
# 데이터셋 불러오는 클래스

class Prompter(object):

    def __init__(self, verbose: bool = False):
        self.template = instruct_template

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:

        if input: # input text가 있다면
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )

        if label:
            res = f"{res}{label}"

        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

prompter = Prompter()

In [ ]:
# Token generation 함수

def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )

    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):

        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"])

    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:

        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"])

        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=add_eos_token)

        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]
    return tokenized_full_prompt

In [ ]:
# 훈련 셋 만들기 (~2분)

val_data = None
train_data = data["train"].shuffle() # random
train_data = train_data.map(generate_and_tokenize_prompt)

### ✅ 'prompt_input' example:

```
아래는 작업을 설명하는 지침과 추가 입력을 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 답변을 작성해주세요.

### 지침:
다음 질문에 답하세요: 제목: 내가 본 최악의 영화 리뷰: 이것은 내가 본 최악의 영화입니다... DVD를 구입하는 유일한 이유는 이 영화를 만든 12살짜리 꼬마가 저능아 제작자에게 팔아넘긴 진짜 감독의 코멘터리를 공개하기 위해서입니다. 가장 좋은 부분은 비디오 게임 클립을 이어 붙인 것을 볼 때였을 것입니다. 이 상품평은 부정적인가요?
답변:

### 입력:
귀하는 지시를 매우 잘 따르는 인공지능 비서입니다. 최대한 많이 도와주세요.

### 답변:
예, 이 제품 리뷰는 부정적입니다.</s>
```

### ✅ 'prompt_no_input' example:

```
아래는 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 답변을 작성해주세요.

### 지침:
x+y = 10$, $2x+y = 13$이 주어졌을 때, $x^2-y^2$를 평가합니다.

### 답변:
x^2-y^2$를 풀려면 $x$와 $y$의 값을 찾아야 합니다. 주어진 방정식을 사용하여 변수 중 하나를 제거하고 다른 변수를 풀 수 있습니다. 첫 번째 방정식을 두 번째 방정식에서 빼면 $2x+y - (x+y) = 13-10$이 나오며, 이는 $x = 3$으로 단순화됩니다. 그런 다음 첫 번째 방정식을 사용하여 $x = 3$을 대입하면 $3+y = 10$을 얻을 수 있으며, 이는 $y = 7$이라는 것을 의미합니다. 이제 $x = 3$과 $y = 7$을 얻었으므로 이를 $x^2-y^2$로 대입할 수 있습니다. 이렇게 하면 $3^2-7^2 = 9-49 = -40$이 됩니다.</s>
```

--------
# 🤗 Apply LoRA config

In [ ]:
# LoRA config 정의

config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM")

In [ ]:
# Model with LoRA

model = prepare_model_for_int8_training(model)
model = get_peft_model(model, config) # Applying LoRA

--------
# ⭐ LLM fine-tuning

In [ ]:
# 만약 이전에 돌렸던 모델을 가져온다면, 아래의 코드 실행

if resume_from_checkpoint:
    checkpoint_name = os.path.join(
        resume_from_checkpoint, "pytorch_model.bin"
    )  # All checkpoint

    if not os.path.exists(checkpoint_name):
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "adapter_model.bin"
        )  # only LoRA model
        resume_from_checkpoint = (
            True
        ) # kyujin: I will use this checkpoint

    if os.path.exists(checkpoint_name):
        print(f"Restarting from {checkpoint_name}")
        adapters_weights = torch.load(checkpoint_name)
        set_peft_model_state_dict(model, adapters_weights)

    else:
        print(f"Checkpoint {checkpoint_name} not found")

In [ ]:
# Trainer class 정의

trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=transformers.TrainingArguments( 
            # 훈련에 이용될 하이퍼파라미터
            per_device_train_batch_size = micro_batch,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_ratio=warmup_ratio,
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=1,
            optim=optimizer,
            evaluation_strategy="no",
            save_strategy="steps",
            neftune_noise_alpha=neftune_noise_alpha,
            max_grad_norm = max_grad_norm,
            save_steps = 13, # you can change!
            lr_scheduler_type=lr_scheduler,
            output_dir=output_dir,
            save_total_limit=2,
            load_best_model_at_end=False,
            ddp_find_unused_parameters=False,
            group_by_length = False
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )

model.config.use_cache = False
model.print_trainable_parameters() # 훈련하는 파라미터의 % 체크

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

In [ ]:
# Training (fine-tuning)
# 훈련시간이 많이 소요됨

torch.cuda.empty_cache()
trainer.train(resume_from_checkpoint=resume_from_checkpoint)

In [ ]:
# 모델 저장

print(f"output_dir: {output_dir}")

model.save_pretrained(output_dir)
model_path = os.path.join(output_dir, "pytorch_model.bin")
torch.save({}, model_path)
tokenizer.save_pretrained(output_dir)

-------
# 🤗 LoRA Adapter merge
> LLM fine-tuning 후, 훈련한 LoRA layer를 기존 base LLM에 붙여서 저장

In [ ]:
# 훈련된 LoRA layer와 base LLM 병합 (merge)

torch.cuda.empty_cache()

base_model = AutoModelForCausalLM.from_pretrained(
    base_LLM_model,
    return_dict = True,
    torch_dtype=torch.float16,
    device_map=device)

model = PeftModel.from_pretrained(base_model, output_dir, device)
model = model.merge_and_unload() # Merge!

In [ ]:
# custom LLM 모델 저장!

final_save_folder = './custom_model'

model.save_pretrained(final_save_folder)
tokenizer.save_pretrained(final_save_folder)

-------
# 🤗 HuggingFace에 모델 업로드

1. 코드에서 Huggingface login 진행
    - HuggingFace에 다시 접속하여, access token 카테고리 접속한다.
    - ⭐`Role: write`로 생성된 token을 복사 한 후, 아래의 `<write_token>` 부분에 붙여넣는다.
 
        ```bash
        !huggingface-cli login --token <write_token>
        ```
        <br>

2. ✅ Huggingface에 접속하여, 모델을 업로드 할 repo를 생성한다.
3. ✅ 아래의 코드를 이용하여 모델과 tokenizer를 업로드한다.
    ```python
    model = AutoModelForCausalLM.from_pretrained(final_save_folder)
    model.push_to_hub("<repo_name>", token=True)
    tokenizer = AutoTokenizer.from_pretrained(final_save_folder)
    tokenizer.push_to_hub("<repo_name>", token=True)
    ```

In [ ]:
# WRITE token
!huggingface-cli login --token <write_token>

In [ ]:
# 모델 업로드 (~10분)
## 먼저, 허깅페이스에 모델 repo를 만든 후 코드를 실행해야 한다!

model = AutoModelForCausalLM.from_pretrained(final_save_folder)
model.push_to_hub("oneonlee/KoSOLAR-v0.2-gugutypus-10.7B", token=True)

In [ ]:
# Tokenizer 업로드

tokenizer = AutoTokenizer.from_pretrained(final_save_folder)
tokenizer.push_to_hub("oneonlee/KoSOLAR-v0.2-gugutypus-10.7B", token=True)

In [ ]:
exit()